# Lesson 2: Interacting with a CSV Data

## Setup and connect to the Azure OpenAI endpoint

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

In [1]:
import os 
import pandas as pd

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain_openai import AzureChatOpenAI

import os
from dotenv import load_dotenv
load_dotenv()

azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")

model = AzureChatOpenAI(
    openai_api_version="2024-04-01-preview",
    azure_deployment=azure_deployment,
    azure_endpoint=azure_endpoint,
    api_key=api_key,
)

## Load the dataset

**Note**: To access the data locally, use the following code:

```
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
```

In [2]:
df = pd.read_csv("./data/all-states-history.csv").fillna(value = 0)

# print sample data
display(df.head())

len(df)


,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-07,AK,305.0,0.0,0,0.0,1293.0,1293.0,33.0,0,...,1731628.0,0,0.0,0.0,0.0,0.0,0.0,0,1731628.0,0
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,0.0,0.0,119757.0,0.0,2323788.0,2347,0.0,0
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,0.0,0.0,0.0,481311.0,0.0,0,2736442.0,3380
3,2021-03-07,AS,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,2140.0,0,0.0,0.0,0.0,0.0,0.0,0,2140.0,0
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,0.0,444089.0,0.0,3842945.0,14856,7908105.0,45110


20780

## Prepare the Langchain dataframe agent

In [3]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import Tool


agent = create_pandas_dataframe_agent(
    llm=model,
    agent_type="tool-calling",
    df=df,
    verbose=True,
    allow_dangerous_code=True,
    max_iterations=100,  # Increase the iteration limit
    max_execution_time=300  # Increase the execution time (in seconds)
)

agent.invoke("how many rows are there? Share python code to you used to find the answer.")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


20780The dataframe `df` contains 20,780 rows. 

Here is the Python code used to find the answer:
```python
len(df)
```

> Finished chain.


{'input': 'how many rows are there? Share python code to you used to find the answer.',
 'output': 'The dataframe `df` contains 20,780 rows. \n\nHere is the Python code used to find the answer:\n```python\nlen(df)\n```'}

## Design your prompt and ask your question

In [30]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer and provide the python code you used.
"""

QUESTION = """How may patients were hospitalized during July 2020 
in state of Texas (state = TX), and nationwide as the total of all states?
"Use the hospitalizedIncrease column""" 


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\npd.set_option('display.max_columns', None)\ndf.head()"}`


         date state    death  deathConfirmed  deathIncrease  deathProbable  \
0  2021-03-07    AK    305.0             0.0              0            0.0   
1  2021-03-07    AL  10148.0          7963.0             -1         2185.0   
2  2021-03-07    AR   5319.0          4308.0             22         1011.0   
3  2021-03-07    AS      0.0             0.0              0            0.0   
4  2021-03-07    AZ  16328.0         14403.0              5         1925.0   

   hospitalized  hospitalizedCumulative  hospitalizedCurrently  \
0        1293.0                  1293.0                   33.0   
1       45976.0                 45976.0                  494.0   
2       14926.0                 14926.0                  335.0   
3           0.0                     0.0                    0.0   
4       57907.0                 579

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients were hospitalized during July 2020 \nin state of Texas (state = TX), and nationwide as the total of all states?\n"Use the hospitalizedIncrease column\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Ans